<a href="https://colab.research.google.com/github/erfanera/SlumImprovment/blob/main/pyngrok_Dharavi_leaf_map_URL(WIP)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pyngrok

In [ ]:
pip install -U geemap

In [ ]:
!pip install streamlit pyngrok geopandas leafmap

In [4]:
from pyngrok import ngrok

# 終止所有现存的 ngrok 会话
ngrok.kill()


In [7]:
from pyngrok import ngrok
import streamlit as st
import geopandas as gpd
import leafmap.foliumap as leafmap
from PIL import Image
import numpy as np
import os
from math import radians, sin, cos, sqrt, atan2
import ee
import geemap as emap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='your-project-id')

# 函数：根据用户输入的坐标创建多边形
def create_polygon_from_bbox(min_point, max_point):
    min_lon, min_lat = min_point
    max_lon, max_lat = max_point
    return [(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat)]

# 函数：创建GeoDataFrame
def create_geodataframe_from_bbox(min_point, max_point):
    rectangle = Polygon(create_polygon_from_bbox(min_point, max_point))
    gdf = gpd.GeoDataFrame([1], geometry=[rectangle], columns=['dummy'])
    return gdf

# 函数：计算地球上两点之间的距离
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth specified in decimal degrees.
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# 函数：计算边界框的尺寸
def get_bbox_dimensions(min_lon, min_lat, max_lon, max_lat):
    """
    Calculate the dimensions of a bounding box.

    Parameters:
    - min_lon (float): Minimum longitude of the bounding box.
    - min_lat (float): Minimum latitude of the bounding box.
    - max_lon (float): Maximum longitude of the bounding box.
    - max_lat (float): Maximum latitude of the bounding box.

    Returns:
    - width (float): The width of the bounding box (longitude span).
    - height (float): The height of the bounding box (latitude span).
    """
    width = max_lon - min_lon
    height = max_lat - min_lat
    return width, height

# 函数：计算边界框尺寸（米）
def get_bbox_dimensions_in_meters(min_lon, min_lat, max_lon, max_lat):
    """
    Calculate the dimensions of a bounding box in meters.

    Parameters:
    - min_lon (float): Minimum longitude of the bounding box.
    - min_lat (float): Minimum latitude of the bounding box.
    - max_lon (float): Maximum longitude of the bounding box.
    - max_lat (float): Maximum latitude of the bounding box.

    Returns:
    - width (float): The width of the bounding box in meters (longitude span).
    - height (float): The height of the bounding box in meters (latitude span).
    """
    avg_lat = (min_lat + max_lat) / 2
    width = haversine(min_lon, avg_lat, max_lon, avg_lat)
    avg_lon = (min_lon + max_lon) / 2
    height = haversine(avg_lon, min_lat, avg_lon, max_lat)
    return width, height

# 函数：将图片划分为网格
def divide_picture_into_grids(width, height, grid_size=1000):
    """
    Divide a picture into grids of one kilometer (1 km) each.

    Parameters:
    - width (float): Width of the picture in meters.
    - height (float): Height of the picture in meters.
    - grid_size (float): Size of each grid cell in meters (default is 1000 meters).

    Returns:
    - list of tuples: Coordinates of the grid cells, each representing a one-kilometer square.
    """
    num_horizontal_grids = int(width / grid_size)
    num_vertical_grids = int(height / grid_size)
    grid_coordinates = []
    for i in range(num_horizontal_grids):
        for j in range(num_vertical_grids):
            grid_x = i * grid_size
            grid_y = j * grid_size
            grid_coordinates.append((grid_x, grid_y))
    return grid_coordinates

# 函数：将图片划分为网格
def split_image_into_grid(image, grid_size):
    img_array = np.array(image)
    height, width = img_array.shape[:2]
    cell_height = height // grid_size
    cell_width = width // grid_size
    grid_images = []
    for i in range(grid_size):
        for j in range(grid_size):
            top = i * cell_height
            bottom = (i + 1) * cell_height
            left = j * cell_width
            right = (j + 1) * cell_width
            grid_cell = image.crop((left, top, right, bottom))
            grid_images.append(grid_cell)
    return grid_images

# Streamlit app
st.title("Custom Bounding Box and Grid Generator")

# 用户输入经纬度坐标
min_lon = st.number_input("Enter minimum longitude:", value=72.8488)
min_lat = st.number_input("Enter minimum latitude:", value=19.0384)
max_lon = st.number_input("Enter maximum longitude:", value=72.8604)
max_lat = st.number_input("Enter maximum latitude:", value=19.0484)

width, height = get_bbox_dimensions_in_meters(min_lon, min_lat, max_lon, max_lat)
st.write(f"Width (Longitude Span): {width} meters")
st.write(f"Height (Latitude Span): {height} meters")

grid_size = st.number_input("Enter grid size (in meters):", value=1000)

# Create bounding box and map
gdf = create_geodataframe_from_bbox((min_lon, min_lat), (max_lon, max_lat))
m = leafmap.Map()
m.add_basemap("SATELLITE")
m.add_gdf(gdf, layer_name="Bounding Box")

# 显示地图
st.write("Here is the bounding box on the map:")
folium_map = m.to_html()
st.write(folium_map, unsafe_allow_html=True)

# 提取地图图像
bbox = [min_lon, min_lat, max_lon, max_lat]
ee_bbox = ee.Geometry.Rectangle(bbox)
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(ee_bbox).first().clip(ee_bbox)
image = image.visualize(bands=['B5', 'B4', 'B3'], min=0, max=0.5)

# 获取地图图像
map_image = emap.ee_to_numpy(image)

# 将图像转换为PIL图像对象
pil_image = Image.fromarray(np.uint8(map_image))

# 划分图片网格
grid_images = split_image_into_grid(pil_image, grid_size)

# 启动 Streamlit 应用
public_url = ngrok.connect(port='8501')
print('Streamlit is running at URL:', public_url)


EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.

In [47]:
%%writefile app.py
import streamlit as st
import geopandas as gpd
from shapely.geometry import Polygon
import leafmap.foliumap as leafmap

# 函数：根据用户输入的坐标创建多边形
def create_polygon_from_bbox(min_point, max_point):
    min_lon, min_lat = min_point
    max_lon, max_lat = max_point
    return [(min_lon, min_lat), (max_lon, min_lat), (max_lon, max_lat), (min_lon, max_lat)]

# 函数：创建GeoDataFrame
def create_geodataframe_from_bbox(min_point, max_point):
    rectangle = Polygon(create_polygon_from_bbox(min_point, max_point))
    gdf = gpd.GeoDataFrame([1], geometry=[rectangle], columns=['dummy'])
    return gdf

# Streamlit 界面设置
st.title("Leafmap and Geopandas in Streamlit")
st.write("Enter the coordinates of the bounding box:")

# 用户输入经纬度坐标
min_lon = st.number_input("Minimum Longitude", value=72.8488)
min_lat = st.number_input("Minimum Latitude", value=19.0384)
max_lon = st.number_input("Maximum Longitude", value=72.8604)
max_lat = st.number_input("Maximum Latitude", value=19.0484)

# 创建地理数据框
pointA = (min_lon, min_lat)
pointB = (max_lon, max_lat)
gdf = create_geodataframe_from_bbox(pointA, pointB)

# 显示地图
m = leafmap.Map()
m.add_basemap("SATELLITE")
m.add_gdf(gdf, layer_name="Bounding Box")
st.write("Here is the bounding box on the map:")
folium_map = m.to_html()
st.components.v1.html(folium_map, height=700)


Overwriting app.py


In [55]:
from pyngrok import ngrok

# 确保没有活动的 ngrok 会话
ngrok.kill()

# 启动 Streamlit 应用
os.system("streamlit run app.py &")

# 建立 ngrok 连接，确保使用正确的参数
public_url = ngrok.connect(addr='8501')
print('Streamlit is running at URL:', public_url)


Streamlit is running at URL: NgrokTunnel: "https://20a6-35-221-22-48.ngrok-free.app" -> "http://localhost:8501"
